In [14]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

def waveform_to_spectrogram(audio_path, output_image_path):
    try:
        # 오디오 파일 로드
        y, sr = librosa.load(audio_path, sr=None)

        # 스펙트로그램 생성
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
        S_dB = librosa.power_to_db(S, ref=np.max)

        # 스펙트로그램 시각화 및 저장
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Mel-frequency spectrogram')
        plt.tight_layout()
        plt.savefig(output_image_path)
        plt.close()

        print(f"Spectrogram image saved successfully at: {output_image_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

# 오디오 파일 및 출력 이미지 경로 설정
audio_path = './takk.wav'
output_image_path = './spectrogram_image.png'

# 파일 존재 여부 확인 및 함수 호출
if os.path.exists(audio_path):
    waveform_to_spectrogram(audio_path, output_image_path)
else:
    print(f"Audio file not found: {audio_path}")

Spectrogram image saved successfully at: ./spectrogram_image.png


In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models

def cnnver(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')  
    ])
    return model

# 입력 이미지의 크기
input_shape = (128, 128, 1)  # 흑백 이미지 (128x128)
model = cnnver(input_shape)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

/Users/gugo-le/Desktop/DEV/python projects/school record/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 859,274 (3.28 MB)

 Trainable params: 859,274 (3.28 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
import os
import shutil

# 디렉토리 생성
data_dir = './data/class1'
os.makedirs(data_dir, exist_ok=True)

# 스펙트로그램 이미지를 생성하거나 이동
source_image_path = './spectrogram_image.png'  # 기존 스펙트로그램 이미지 경로
destination_image_path = os.path.join(data_dir, 'spectrogram_image.png')

if os.path.exists(source_image_path):
    shutil.move(source_image_path, destination_image_path)
else:
    print(f"Source image not found: {source_image_path}")

print(f"Image successfully moved to: {destination_image_path}")

Image successfully moved to: ./data/class1/spectrogram_image.png


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 증강 및 분할
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    './data',  
    target_size=(128, 128),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical', 
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    './data', 
    target_size=(128, 128),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

# 클래스 개수 확인
num_classes = len(train_generator.class_indices)

# 모델 정의
model = Sequential([
    Flatten(input_shape=(128, 128, 1)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# 모델 학습
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)